In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from array import array
from get_cursor import get_cursor
import statements

curs, db = get_cursor('../me.db')

In [ ]:
# get interface to sqlite database, column names
info = [curs.execute('pragma table_info(Daily_Functionality);').fetchall()][0]
labels = [x[1] for x in info]


In [ ]:
columns = {}
for a in labels:
    columns[a] = curs.execute('select Date, ' + a + ' from Daily_Functionality;').fetchall()

# columns is a dictionary 
# in which values are lists of 420 2-string tuples:
# columns[key] = [ 420*(<date_string>, <number_string>) ]
# for each key

In [ ]:
dates = [d[0] for d in columns['Date']]
for k in columns.keys():
    temp_cols = []
    for m in range(len(columns[k])):
        try:
            temp_cols.append(float(columns[k][m][1]))
        except (TypeError, ValueError):
            pass
    #
    try:
        assert type(temp_cols[1]) == float
    except (AssertionError, IndexError):
        continue
    r = np.array(temp_cols)
    print(k + ': ')
    fig = plt.figure()
    ax = fig.add_axes([0, 0, len(dates)/40, 4])
    ax.plot(list(range(len(dates))), r)
    ax.set_title(k)
    plt.show()
    print('\tAverage:', str(np.average(r)))
    print('\tVariance:', str(r.var()))
    print('\tStandard deviation:', str(r.std()))
    #
    columns[k] = r

In [ ]:
for k in columns.keys():
    print('\n=================================================\n')
    print(k.upper())
    for j in columns.keys():
        if j != k:
            try:
                # These don't appear to be the functions I want...
                print('\n\t' + k + ' & ' + j + ' correlation: ' + str(np.corrcoef(columns[k], columns[j])[0][1]))
                #print(str(np.correlate(columns[k], columns[j])))
                print('\n\t' + k + ' & ' + j + ' covariance:' + str(np.cov(columns[k], columns[j])[0][1]))
                #print(str(np.cov(columns[k], columns[j])))
                print('\n\t\t=====================\n')
            except Exception as e:
                print(e)